In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
from time import time
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine

load_dotenv(".env")

# user = os.environ['POSTGRES_USER']
# password = os.environ['POSTGRES_PASSWORD']
# host = os.environ['POSTGRES_HOST']
# port = os.environ['POSTGRES_PORT']
# db = os.environ['POSTGRES_DBNAME']
# table_name = os.environ['POSTGRES_SCHEMA']

user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
db = 'postgres'
table_name = 'order_data'
schema = 'public'


engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')


order_data_iter = pd.read_csv("../data/order_data.csv", iterator=True, chunksize=100000)

print(host, port, db, table_name)

order_data_df = next(order_data_iter)

order_data_df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')

order_data_df.to_sql(name=table_name, con=engine, if_exists='append')


while True: 

    try:
        t_start = time()
        
        df = next(order_data_iter)

        df.to_sql(name=table_name, con=engine, if_exists='replace')

        t_end = time()

        print('inserted another chunk, took %.3f second' % (t_end - t_start))

    except StopIteration:
        print("Finished ingesting data into the postgres database")
        break


localhost 5432 postgres order_data
Finished ingesting data into the postgres database


In [14]:
query = """
SELECT * FROM marketing_data LIMIT 10
"""

pd.read_sql(query, con=engine)

,index,Campaign_ID,Target_Audience,Campaign_Goal,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,Product
0,200000,196682,Men 45-60,Product Launch,45 Days,Facebook,0.08,$10175.50,3.050000,New York,English,26833,81000,7,Home,2022-05-17,Laptop
1,200001,865386,Women 45-60,Increase Sales,45 Days,Twitter,0.13,$10175.51,1.240000,New York,Spanish,26833,81000,8,Food,2022-05-31,Smartes Thermostat
2,200002,868814,Men 35-44,Brand Awareness,45 Days,Instagram,0.05,$10175.56,4.500000,Los Angeles,English,26833,81001,5,Home,2022-09-15,None
3,200003,790730,Men 35-44,Brand Awareness,45 Days,Pinterest,0.03,$10175.58,1.118958,Austin,French,15759,52297,1,Fashion,2022-11-02,Soundbar
4,200004,456762,Women 25-34,Product Launch,45 Days,Pinterest,0.12,$10175.68,1.061759,Los Angeles,French,15759,52297,1,Health,2022-01-08,Action-Kamera
5,200005,442131,Men 35-44,Brand Awareness,45 Days,Pinterest,0.12,$10175.74,0.972385,Austin,English,15760,52298,1,Health,2022-05-30,Powerbank
6,200006,200305,Women 25-34,Brand Awareness,45 Days,Instagram,0.05,$10175.75,5.690000,Los Angeles,French,26834,81002,1,Home,2022-03-22,Laptop
7,200007,304696,Women 45-60,Increase Sales,45 Days,Facebook,0.10,$10175.79,5.270000,Austin,English,26834,81002,2,Food,2022-09-25,Smarte Glühbirnen
8,200008,885890,Women 18-24,Product Launch,45 Days,Facebook,0.01,$10175.80,0.000000,New York,English,26834,81003,8,Health,2022-09-16,Netzteil
9,200009,563593,Men 35-44,Brand Awareness,45 Days,Twitter,0.06,$10175.87,0.140000,New York,Spanish,26834,81003,8,Health,2022-02-02,Action-Kamera
